In [20]:
import pandas as pd
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
from sklearn.preprocessing import Imputer
from sklearn import preprocessing
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
import xgboost 

In [21]:
df_train=pd.read_csv('/home/luffy/Documents/Titanic/train.csv')
df_test=pd.read_csv('/home/luffy/Documents/Titanic/test.csv')


In [22]:
Y_train=df_train.Survived

X_train=df_train.drop(['PassengerId','Name','Ticket','Cabin','Survived'],axis=1)
X_test=df_test.drop(['PassengerId','Name','Ticket','Cabin'],axis=1)

In [23]:
le = preprocessing.LabelEncoder()
encoded=le.fit_transform(df_train['Sex'].astype('str'))
X_train['Sex']=encoded

encoded=le.fit_transform(df_train['Embarked'].astype('str'))
X_train['Embarked']=encoded

In [24]:
X_test['Sex']=le.fit_transform(df_test['Sex'].astype('str'))
X_test['Embarked']=le.fit_transform(df_test['Embarked'].astype('str'))

In [25]:
impute=Imputer(strategy='mean')
for col in X_train.columns:
    if X_train[col].isnull().any():
        imputed=pd.DataFrame(impute.fit_transform(X_train[[col]]))
        X_train[[col]]=imputed
print(X_train.dtypes)

Pclass        int64
Sex           int64
Age         float64
SibSp         int64
Parch         int64
Fare        float64
Embarked      int64
dtype: object


In [26]:
for col in X_test.columns:
    if X_test[col].isnull().any():
        imputed=pd.DataFrame(impute.fit_transform(X_test[[col]]))
        X_test[[col]]=imputed
print(X_test.isnull().sum())

Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64


In [27]:
models = []
models.append(('LR', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC()))
models.append(('RFC',RandomForestClassifier()))
models.append(('ETC',ExtraTreesClassifier()))
models.append(('XGB',xgboost.XGBClassifier()))
# evaluate each model in turn
results = []
names = []
for name, model in models:
	kfold = model_selection.KFold(n_splits=10, random_state=7)
	cv_results = model_selection.cross_val_score(model, X_train, Y_train, cv=kfold, scoring='accuracy')
	results.append(cv_results)
	names.append(name)
	msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
	print(msg)

LR: 0.793508 (0.031727)
LDA: 0.793508 (0.033281)
KNN: 0.704881 (0.037765)
CART: 0.776667 (0.025729)
NB: 0.789051 (0.022893)
SVM: 0.697079 (0.060606)
RFC: 0.800287 (0.033141)
ETC: 0.782310 (0.035285)


/home/luffy/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/luffy/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/luffy/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/luffy/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambig

XGB: 0.822709 (0.034134)


/home/luffy/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/luffy/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/luffy/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [28]:
xgb=xgboost.XGBClassifier(n_estimators=50)

In [29]:
xgb.fit(X_train,Y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=50,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [30]:
prediction=pd.DataFrame(xgb.predict(X_test))

/home/luffy/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [31]:
print(prediction)

     0
0    0
1    0
2    0
3    0
4    0
5    0
6    1
7    0
8    1
9    0
10   0
11   0
12   1
13   0
14   1
15   1
16   0
17   0
18   1
19   0
20   0
21   1
22   1
23   1
24   1
25   0
26   1
27   0
28   0
29   0
..  ..
388  0
389  0
390  0
391  1
392  0
393  0
394  0
395  1
396  0
397  1
398  0
399  0
400  1
401  0
402  1
403  0
404  0
405  0
406  0
407  0
408  1
409  1
410  1
411  1
412  1
413  0
414  1
415  0
416  0
417  0

[418 rows x 1 columns]


In [32]:
sol=pd.DataFrame({'PassengerId':range(892,1310,1)})
sol['Survived']=prediction
sol.to_csv('/home/luffy/Documents/Titanic/Titanic_solution.csv')